<center><h1>Management and Analysis of Physics Dataset (MOD. B) </h1></center>
<center><h2> Project 5 - Streaming processing of cosmic rays using Drift Tubes detectors</h2></center>
<center><h2>Group 2305</h2></center>

<center><style>
    table {font-size: 24px;}
</style></center>

| Last Name        | First Name         |Student ID|
|:----------------:|:------------------:|:--------------:|
| Bertinelli       | Gabriele           |1219907 (tri)   |
| Bhatti           | Roben              |2091187         |
| Bonato           | Diego              |2091250         |
| Cacciola         | Martina            |2097476         |

<left><h2> Part 4 - Metrics analysis</h2></left>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import json
import csv
import seaborn as sns
import os
import re

## Best configuration

In [2]:
# Specify the target filename
target_filename = '10ex_1core_10sp_aT_3w_10kp.json'

df1_list = []

for filename in os.listdir('./metriche/'):
    if filename == target_filename:
        # Retrieve the file path
        file_path = os.path.join('./metriche/', filename)

        # Extract the parameters from the filename
        params = filename.split('.')[0].split('_')
        ex = int(params[0].replace('ex', ''))
        core = int(params[1].replace('core', ''))
        sp = int(params[2].replace('sp', ''))
        arrow = params[3]
        w = int(re.findall(r'\d+', params[4])[0])
        kp = int(re.findall(r'\d+', params[5])[0]) if len(params) > 5 and params[5].endswith('kp') else None
        batch = int(re.findall(r'\d+', params[6])[0]) if len(params) > 6 and params[6].startswith('batch') else 1000
        secProcTime = int(re.findall(r'\d+', params[-1])[0]) if params[-1].endswith('secProcTime') else 5

        with open(file_path, 'r') as f:
            data = json.load(f)

        df1 = pd.DataFrame(data)
        df1['Executors'] = ex
        df1['Cores'] = core
        df1['ShufflePartitions'] = sp
        df1['Arrow'] = arrow
        df1['Workers'] = w
        df1['KafkaPartitions'] = kp
        df1['Batches'] = batch
        df1['Trigger'] = secProcTime

        batchId = []
        numInputRows = []
        inputRowsPerSecond = []
        processedRowsPerSecond = []
        durationMs = []
        numOutputRows = []

        for item in data:
            try:
                batchId.append(item["batchId"])
                numInputRows.append(item["numInputRows"])
                inputRowsPerSecond.append(item["inputRowsPerSecond"])
                processedRowsPerSecond.append(item["processedRowsPerSecond"])
                duration_Ms = item["durationMs"]["triggerExecution"] / 1000  # convert from ms to s
                durationMs.append(duration_Ms)
                numOutputRows.append(item["sink"]["numOutputRows"])
            except Exception as e:
                batchID_debug = item["batchId"]
                print(f"Error encountered in batch {batchID_debug}: {e}")
                continue

        # Check and align the lengths of the arrays
        length = max(len(batchId), len(numInputRows), len(inputRowsPerSecond),
                     len(processedRowsPerSecond), len(durationMs), len(numOutputRows))

        batchId += [np.nan] * (length - len(batchId))
        numInputRows += [np.nan] * (length - len(numInputRows))
        inputRowsPerSecond += [np.nan] * (length - len(inputRowsPerSecond))
        processedRowsPerSecond += [np.nan] * (length - len(processedRowsPerSecond))
        durationMs += [np.nan] * (length - len(durationMs))
        numOutputRows += [np.nan] * (length - len(numOutputRows))

        # Add arrays as columns to the DataFrame
        df1['batchId'] = batchId
        df1['numInputRows'] = numInputRows
        df1['inputRowsPerSecond'] = inputRowsPerSecond
        df1['processedRowsPerSecond'] = processedRowsPerSecond
        df1['durationMs'] = durationMs
        df1['numOutputRows'] = numOutputRows

        # Compute message rate
        df1['msgrate'] = df1['numInputRows'] / (df1['durationMs'] / 1000)  # Convert duration to seconds

        # Drop the unnecessary columns
        drop_columns = ['id', 'runId', 'name', 'timestamp', 'stateOperators', 'sources', 'sink', 'numOutputRows']
        df1 = df1.drop(drop_columns, axis=1, errors='ignore')

        # Append DataFrame to the list
        df1_list.append(df1)

# Concatenate all DataFrames in the list
df1 = pd.concat(df1_list, ignore_index=True)
df1

,batchId,numInputRows,inputRowsPerSecond,processedRowsPerSecond,durationMs,Executors,Cores,ShufflePartitions,Arrow,Workers,KafkaPartitions,Batches,Trigger,msgrate
0,0,0,0.000000,0.000000,31.751,10,1,10,aT,3,10,1000,5,0.000000e+00
1,1,28000,880.669309,1421.608448,19.695,10,1,10,aT,3,10,1000,5,1.421681e+06
2,2,18000,913.751967,1643.385374,10.953,10,1,10,aT,3,10,1000,5,1.643385e+06
3,3,10000,912.741877,1282.873637,7.795,10,1,10,aT,3,10,1000,5,1.282874e+06
4,4,7000,897.666068,1013.024602,6.910,10,1,10,aT,3,10,1000,5,1.013025e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,307,5000,1000.000000,1881.821603,2.657,10,1,10,aT,3,10,1000,5,1.881822e+06
308,308,4000,800.000000,1674.340728,2.389,10,1,10,aT,3,10,1000,5,1.674341e+06
309,309,5000,1000.000000,2000.800320,2.499,10,1,10,aT,3,10,1000,5,2.000800e+06
310,310,4650,930.000000,1746.806912,2.662,10,1,10,aT,3,10,1000,5,1.746807e+06


## Worst configuration

In [3]:
# Specify the target filename
target_filename = '10ex_1core_10sp_aT_3w_200kp_5000batch_5secProcTime.json'

df2_list = []

for filename in os.listdir('./metriche/'):
    if filename == target_filename:
        # Retrieve the file path
        file_path = os.path.join('./metriche/', filename)

        # Extract the parameters from the filename
        params = filename.split('.')[0].split('_')
        ex = int(params[0].replace('ex', ''))
        core = int(params[1].replace('core', ''))
        sp = int(params[2].replace('sp', ''))
        arrow = params[3]
        w = int(re.findall(r'\d+', params[4])[0])
        kp = int(re.findall(r'\d+', params[5])[0]) if len(params) > 5 and params[5].endswith('kp') else None
        batch = int(re.findall(r'\d+', params[6])[0]) if len(params) > 6 and params[6].startswith('batch') else 1000
        secProcTime = int(re.findall(r'\d+', params[-1])[0]) if params[-1].endswith('secProcTime') else 5

        with open(file_path, 'r') as f:
            data = json.load(f)

        df2 = pd.DataFrame(data)
        df2['Executors'] = ex
        df2['Cores'] = core
        df2['ShufflePartitions'] = sp
        df2['Arrow'] = arrow
        df2['Workers'] = w
        df2['KafkaPartitions'] = kp
        df2['Batches'] = batch
        df2['Trigger'] = secProcTime

        batchId = []
        numInputRows = []
        inputRowsPerSecond = []
        processedRowsPerSecond = []
        durationMs = []
        numOutputRows = []

        for item in data:
            try:
                batchId.append(item["batchId"])
                numInputRows.append(item["numInputRows"])
                inputRowsPerSecond.append(item["inputRowsPerSecond"])
                processedRowsPerSecond.append(item["processedRowsPerSecond"])
                duration_Ms = item["durationMs"]["triggerExecution"] / 1000  # convert from ms to s
                durationMs.append(duration_Ms)
                numOutputRows.append(item["sink"]["numOutputRows"])
            except Exception as e:
                batchID_debug = item["batchId"]
                print(f"Error encountered in batch {batchID_debug}: {e}")
                continue

        # Check and align the lengths of the arrays
        length = max(len(batchId), len(numInputRows), len(inputRowsPerSecond),
                     len(processedRowsPerSecond), len(durationMs), len(numOutputRows))

        batchId += [np.nan] * (length - len(batchId))
        numInputRows += [np.nan] * (length - len(numInputRows))
        inputRowsPerSecond += [np.nan] * (length - len(inputRowsPerSecond))
        processedRowsPerSecond += [np.nan] * (length - len(processedRowsPerSecond))
        durationMs += [np.nan] * (length - len(durationMs))
        numOutputRows += [np.nan] * (length - len(numOutputRows))

        # Add arrays as columns to the DataFrame
        df2['batchId'] = batchId
        df2['numInputRows'] = numInputRows
        df2['inputRowsPerSecond'] = inputRowsPerSecond
        df2['processedRowsPerSecond'] = processedRowsPerSecond
        df2['durationMs'] = durationMs
        df2['numOutputRows'] = numOutputRows

        # Compute message rate
        df2['msgrate'] = df2['numInputRows'] / (df2['durationMs'] / 1000)  # Convert duration to seconds

        # Drop the unnecessary columns
        drop_columns = ['id', 'runId', 'name', 'timestamp', 'stateOperators', 'sources', 'sink', 'numOutputRows']
        df2 = df2.drop(drop_columns, axis=1, errors='ignore')

        # Append DataFrame to the list
        df2_list.append(df2)

# Concatenate all DataFrames in the list
df2 = pd.concat(df2_list, ignore_index=True)
df2

,batchId,numInputRows,inputRowsPerSecond,processedRowsPerSecond,durationMs,Executors,Cores,ShufflePartitions,Arrow,Workers,KafkaPartitions,Batches,Trigger,msgrate
0,0,275,0.000000,6.926080,39.703,10,1,10,aT,3,200,1000,5,6.926429e+03
1,1,62515,1571.557857,3216.785016,19.433,10,1,10,aT,3,200,1000,5,3.216951e+06
2,2,30506,1569.319409,2212.343172,13.789,10,1,10,aT,3,200,1000,5,2.212343e+06
3,3,22459,1628.407773,1716.392816,13.085,10,1,10,aT,3,200,1000,5,1.716393e+06
4,4,21248,1623.471883,1866.479269,11.384,10,1,10,aT,3,200,1000,5,1.866479e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,108,11676,1490.045942,1731.060044,6.745,10,1,10,aT,3,200,1000,5,1.731060e+06
109,109,9711,1439.093065,1401.703233,6.928,10,1,10,aT,3,200,1000,5,1.401703e+06
110,110,10560,1523.809524,1537.117904,6.870,10,1,10,aT,3,200,1000,5,1.537118e+06
111,111,10274,1494.834861,1376.289350,7.465,10,1,10,aT,3,200,1000,5,1.376289e+06


## Creating the metrics dataframe

In [4]:
df_list = []
labels = ['a','b','c','d','e','f','g','h','i','l','m','n','o','p','q','r','s','t','u','v','z']

for index, filename in enumerate(os.listdir('./metriche/')):
    if filename.endswith('.json'):
        params = filename.split('.')[0].split('_')
        #label = labels[index % len(labels)]

        ex = int(params[0].replace('ex', ''))
        core = int(params[1].replace('core', ''))
        sp = int(params[2].replace('sp', ''))
        arrow = params[3]
        w = int(re.findall(r'\d+', params[4])[0])
        kp = int(re.findall(r'\d+', params[5])[0]) if len(params) > 5 and params[5].endswith('kp') else None
        batch = int(re.findall(r'\d+', params[6])[0]) if len(params) > 6 and params[6].startswith('batch') else 1000
        secProcTime = int(re.findall(r'\d+', params[-1])[0]) if params[-1].endswith('secProcTime') else 5

        with open('./metriche/' + filename, 'r') as f:
            data = json.load(f)

        df = pd.DataFrame(data)
        df['Label']=label
        df['Executors'] = ex
        df['Cores'] = core
        df['ShufflePartitions'] = sp
        df['Arrow'] = arrow
        df['Workers'] = w
        df['KafkaPartitions']=kp
        df['Batches'] = batch
        df['Trigger'] = secProcTime

        batchId = []
        numInputRows = []
        inputRowsPerSecond = []
        processedRowsPerSecond = []
        durationMs = []
        numOutputRows = []

        for item in data:
            try:
                batchId.append(item["batchId"])
                numInputRows.append(item["numInputRows"])
                inputRowsPerSecond.append(item["inputRowsPerSecond"])
                processedRowsPerSecond.append(item["processedRowsPerSecond"])

                duration_Ms = (
                     item["durationMs"]["triggerExecution"]/1000 #convert from ms to s
                )
                durationMs.append(duration_Ms)

                numOutputRows.append(item["sink"]["numOutputRows"])
            except Exception as e:
                batchID_debug = item["batchId"]
                print(f"Error encountered in batch {batchID_debug}: {e}")
                continue

        # Check and align the lengths of the arrays
        length = max(len(batchId), len(numInputRows), len(inputRowsPerSecond),
                     len(processedRowsPerSecond), len(durationMs), len(numOutputRows))

        batchId += [np.nan] * (length - len(batchId))
        numInputRows += [np.nan] * (length - len(numInputRows))
        inputRowsPerSecond += [np.nan] * (length - len(inputRowsPerSecond))
        processedRowsPerSecond += [np.nan] * (length - len(processedRowsPerSecond))
        durationMs += [np.nan] * (length - len(durationMs))
        numOutputRows += [np.nan] * (length - len(numOutputRows))

         # Add arrays as columns to the DataFrame
        df['batchId'] = batchId
        df['numInputRows'] = numInputRows
        df['inputRowsPerSecond'] = inputRowsPerSecond
        df['processedRowsPerSecond'] = processedRowsPerSecond
        df['durationMs'] = durationMs
        df['numOutputRows'] = numOutputRows

        # Compute message rate
        df['msgrate'] = df['numInputRows'] / (df['durationMs'] / 1000)  # Convert duration to
        # Drop the unnecessary columns 
        drop_columns = ['id', 'runId', 'name', 'timestamp', 'stateOperators', 'sources', 'sink', 'numOutputRows']
        df = df.drop(drop_columns, axis=1, errors='ignore')

        # Append DataFrame to the list
        df_list.append(df)

# Concatenate all DataFrames in the list
df = pd.concat(df_list, ignore_index=True)
df

NameError: name 'label' is not defined

## Plot batch processing time

In [ ]:
#comparing best vs worst configuration
#worst config: 10 ex, 1 core, 50 sp, 3 w, 1000 rows/sec, 5 sec of trigger

# Plotting the single file
batch_durations_best = df1['durationMs']
mean_batch_best = batch_durations_best[25:-1].mean()

batch_durations_worst = df2['durationMs']
mean_batch_worst = batch_durations_worst[25:-1].mean()

fig, ax = plt.subplots(figsize=(12, 4), dpi=500)
ax.plot(df1['batchId'], batch_durations_best, label='best_config')
ax.plot(df2['batchId'], batch_durations_worst, label='worst_config')

ax.set_xlabel('Batch #', fontsize=14)
ax.set_ylabel('Batch processing time [s]', fontsize=14)
ax.set_title('Best vs Worst Configuration')
ax.set_xlim(-1, 210)
ax.plot([],[], ' ', label=f'Avg time best={mean_batch_best:1.3f} s')
ax.plot([],[], ' ', label=f'Avg time worst={mean_batch_worst:1.3f} s')
plt.legend()

# Create zoom inset
axins = ax.inset_axes([0.21, 0.42, 0.54, 0.30])
axins.plot(df1['batchId'][28:82], batch_durations_best[28:82])
axins.plot(df2['batchId'][28:82], batch_durations_worst[28:82])
axins.set_ylim(1, 9.5)

# Add border around the zoom inset
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

plt.show()

In [ ]:
stop here

## Batch processing time

In [ ]:
#using best configuration: 10 ex, 1 core, 1 sp, aT, 3 w, 10 kp, 5 sec of trigger, 1000 rows/sec

target_label_best = 'd'

# Filter the DataFrame based on the target label
df_target_best = df[df['Label'] == target_label_best]

# Plotting the single file
batch_durations_best = df_target_best['durationMs']
mean_batch_best = batch_durations_best[50:-1].mean()              #restrict the computation to the plateau

fig, ax = plt.subplots(figsize=(12, 4), dpi=500)
ax.plot(df_target_best['batchId'], batch_durations_best)

ax.set_xlabel('Batch #', fontsize=14)
ax.set_ylabel('Batch processing time [s]', fontsize=14)
ax.set_title('Best Configuration')
ax.plot([],[], ' ', label=f'Avg time={mean_batch_best:1.3f} s')
plt.legend()

# Create zoom inset
axins = ax.inset_axes([0.33, 0.42, 0.6, 0.35])
axins.plot(df_target_best['batchId'][75:200], batch_durations_best[75:200])


# Add border around the zoom inset
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

plt.show()

In [ ]:
df[df['Label']=='n']

In [ ]:
#comparing best vs worst configuration
#worst config: 10 ex, 1 core, 50 sp, 3 w, 1000 rows/sec, 5 sec of trigger

target_label_best = 'd'
target_label_worst = 'n'

# Filter the DataFrame based on the target label
df_target_best = df[df['Label'] == target_label_best]
df_target_worst = df[df['Label'] == target_label_worst]

# Plotting the single file
batch_durations_best = df_target_best['durationMs']
mean_batch_best = batch_durations_best[50:-1].mean()

batch_durations_worst = df_target_worst['durationMs']
mean_batch_worst = batch_durations_worst[50:-1].mean()

fig, ax = plt.subplots(figsize=(12, 4), dpi=500)
ax.plot(df_target_best['batchId'], batch_durations_best, label='best_config')
ax.plot(df_target_worst['batchId'], batch_durations_worst, label='worst_config')

ax.set_xlabel('Batch #', fontsize=14)
ax.set_ylabel('Batch processing time [s]', fontsize=14)
ax.set_title('Best vs Worst Configuration')
ax.set_xlim(-1, 200)
ax.plot([],[], ' ', label=f'Avg time best={mean_batch_best:1.3f} s')
ax.plot([],[], ' ', label=f'Avg time worst={mean_batch_worst:1.3f} s')
plt.legend()

# Create zoom inset
axins = ax.inset_axes([0.21, 0.42, 0.54, 0.30])
axins.plot(df_target_best['batchId'][25:115], batch_durations_best[25:115])
axins.plot(df_target_worst['batchId'][25:115], batch_durations_worst[25:115])
axins.set_ylim(1, 7)

# Add border around the zoom inset
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

plt.show()

## Input and Processed Rows per Second

We plot the input and processed rows per second for two settings with different batch size but keeping the rest of the parameters fixed: 10 executors, 1 core, 10 shuffle partition, arrow = True, 3 workers, 10 Kafka partitions, 5 sec of trigger.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 10), dpi=500)

#selecting the labels corresponding to 1000 and 5000 rows/s
target_label_5000 = 'n'
target_label_1000 = 'd'

# Filter the DataFrame based on the target label
df_target_5000 = df[df['Label'] == target_label_5000]
df_target_1000 = df[df['Label'] == target_label_1000]

mean_in_1000 = df_target_1000['inputRowsPerSecond'][50:-1].mean()
mean_out_1000 = df_target_1000['processedRowsPerSecond'][50:-1].mean()

mean_in_5000 = df_target_5000['inputRowsPerSecond'][50:-1].mean()
mean_out_5000 = df_target_5000['processedRowsPerSecond'][50:-1].mean()

ax[0].plot(df_target_1000['batchId'][0:175], df_target_1000['inputRowsPerSecond'][0:175], color='xkcd:dull blue', label='Input Rows/s')
ax[0].plot(df_target_1000['batchId'][0:175], df_target_1000['processedRowsPerSecond'][0:175], color='green', label='Processed Rows/s')
ax[0].set_xlabel('Batch #', fontsize=14)
ax[0].set_ylabel('rows/s', fontsize=14)
ax[0].set_title('1000 rows/s', fontsize=14)  # Set the custom title
ax[0].plot([],[], ' ', label=f'Avg input rows={mean_in_1000:1.3f}/s')
ax[0].plot([],[], ' ', label=f'Avg processed rows={mean_out_1000:1.3f}/s')
ax[0].legend()

ax[1].plot(df_target_5000['batchId'], df_target_5000['inputRowsPerSecond'], color='xkcd:dull blue', label='Input Rows/s')
ax[1].plot(df_target_5000['batchId'], df_target_5000['processedRowsPerSecond'], color='green', label='Processed Rows/s')
ax[1].set_xlabel('Batch #', fontsize=14)
ax[1].set_ylabel('rows/s', fontsize=14)
ax[1].set_title('5000 rows/s', fontsize=14)  # Set the custom title
ax[1].plot([],[], ' ', label=f'Avg input rows={mean_in_5000:1.3f}/s')
ax[1].plot([],[], ' ', label=f'Avg processed rows={mean_out_5000:1.3f}/s')
ax[1].legend()

plt.show()

## Box Plot of Duration Time

We plot the duration time by varying the number of Kafka partitions and keeping the number of shuffle partitions fixed, and vice versa.

### Varying Kafka Partitions

In [ ]:
# Define the criteria
criteria = {
    'Executors': 10,
    'Cores': 1,
    'ShufflePartitions': 10,
    'Trigger': 5,
    'Workers': 3,
    'Arrow': 'aT',
    'Batches': 5000
}

# Convert the 'KafkaPartitions' column to categorical type
df['KafkaPartitions'] = df['KafkaPartitions'].astype('category')

filtered_df = df[df['KafkaPartitions'] != 6]
unique_partitions = filtered_df['KafkaPartitions'].unique()

# Create the box plot using seaborn
plt.figure(figsize=(12, 8), dpi=500)
sns.boxplot(data=filtered_df[50:-1], y=filtered_df['KafkaPartitions'].cat.remove_unused_categories(), x='durationMs', showfliers=False, orient='h')
plt.ylabel('Kafka Partition')
plt.xlabel('Duration [s]')
plt.title('Duration Time per Kafka Partition')

plt.show()

### Varying Shuffle Partitions

In [ ]:
# Define the criteria
criteria = {
    'Executors': 10,
    'Cores': 1,
    'KafkaPartitions': 10,
    'Trigger': 5,
    'Workers': 3,
    'Arrow': 'aT',
    'Batches': 5000
}

# Convert the 'ShufflePartitions' column to categorical type
df['ShufflePartitions'] = df['ShufflePartitions'].astype('category')

filtered_df = df[df['ShufflePartitions'] != 6]
unique_partitions = filtered_df['ShufflePartitions'].unique()

# Create the box plot using seaborn
plt.figure(figsize=(12, 8), dpi=500)
sns.boxplot(data=filtered_df[50:-1], y=filtered_df['ShufflePartitions'].cat.remove_unused_categories(), x='durationMs', showfliers=False, orient='h')
plt.ylabel('Shuffle Partition')
plt.xlabel('Duration [s]')
plt.title('Duration Time per Shuffle Partition')

plt.show()